In [3]:
import sys
sys.path.append('/home/cocoza4/workspace/absorouteio/asr-face-recognition/src')

import numpy as np
import tensorflow as tf
from PIL import Image

from models import create_model

tf.__version__

'2.1.0'

In [4]:
emb_size = 10
n_classes = 20

In [34]:
model = create_model('seresnet152', emb_size)
model

In [35]:
img = Image.open('/home/cocoza4/datasets/asr_plastic/train/IMG_20200222_011937.jpg').resize((112, 112))
img = np.asarray(img).astype('float32') / 255.
inputs = np.array([img, img])
inputs.shape

(2, 112, 112, 3)

In [36]:
model(inputs)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.0036821 , -0.02201095, -0.01564291,  0.00357328, -0.01834397,
        -0.00714342,  0.00014136, -0.0334766 ,  0.03271113,  0.03801651],
       [-0.00368211, -0.02201094, -0.0156429 ,  0.0035733 , -0.01834397,
        -0.00714341,  0.00014138, -0.0334766 ,  0.03271111,  0.03801651]],
      dtype=float32)>

# ResNet100

In [3]:
units = [3, 13, 30, 3]
filter_list = [64, 64, 128, 256, 512]
bottle_neck = False
num_stages = 4


version_se = net_se = 0
net_act = 'prelu'
version_input = net_input = 1
net_unit = 1
version_unit = 3
fc_type = 'E'
num_unit = len(units)
dim_match = False

In [4]:
import mxnet as mx
from mxnet import nd

def mx_execute(symbol, tensors):
    shapes = {n: t.shape for n, t in tensors.items()}
    executor = symbol.simple_bind(ctx=mx.cpu(), **shapes)
    return executor.forward(**tensors)

In [5]:
inp = np.random.normal(size=(2, 3, 200, 200)).astype('float32')
tf_inp = np.transpose(inp, (0, 3, 2, 1))
inp.shape

(2, 3, 200, 200)

In [6]:
inp.sum(), tf_inp.sum()

(590.8825, 590.8825)

In [7]:
mx_inp = nd.from_numpy(inp)
mx_inp.ndim

4

In [8]:
mx_inp.asnumpy().shape

(2, 3, 200, 200)

In [9]:
xxx = mx.sym.Variable('embedding')
pool = mx.sym.Pooling(data=xxx, global_pool=True, kernel=(7, 7), pool_type='avg', name='pool')
pool

<Symbol pool>

In [80]:
mx_execute(pool, tensors={'embedding': mx_inp.asnumpy()})

[
 [[[[-0.00708314]]
 
   [[-0.00617472]]
 
   [[ 0.00464411]]]
 
 
  [[[ 0.0024788 ]]
 
   [[ 0.0151381 ]]
 
   [[ 0.00024123]]]]
 <NDArray 2x3x1x1 @cpu(0)>]

In [81]:
MeanLayer = tf.keras.layers.Lambda(lambda x: tf.reduce_mean(x, axis=[1, 2], keepdims=True))

In [82]:
MeanLayer(tf_inp)

<tf.Tensor: shape=(2, 1, 1, 3), dtype=float32, numpy=
array([[[[-0.00708316, -0.00617473,  0.00464414]]],


       [[[ 0.0024788 ,  0.01513811,  0.00024121]]]], dtype=float32)>

In [83]:
tf.keras.layers.GlobalAveragePooling2D()(tf_inp)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[-0.00708316, -0.00617473,  0.00464414],
       [ 0.0024788 ,  0.01513811,  0.00024121]], dtype=float32)>

In [84]:
num_filter = 64
stride = strides = (2, 2)

def Conv(**kwargs):
    body = mx.sym.Convolution(**kwargs)
    return body

In [85]:
data = mx.sym.Variable('data')
bn1 = mx.sym.BatchNorm(data=data, fix_gamma=False, eps=2e-5,)
conv1 = Conv(data=bn1, num_filter=num_filter, kernel=(3,3), stride=(1,1), pad=(1,1),
                              no_bias=True)
bn2 = mx.sym.BatchNorm(data=conv1, fix_gamma=False, eps=2e-5)
# act1 = Act(data=bn2, act_type=act_type, name=name + '_relu1')
act1 = mx.sym.LeakyReLU(data=bn2, act_type='prelu')
conv2 = Conv(data=act1, num_filter=num_filter, kernel=(3,3), stride=stride, pad=(1,1),
                              no_bias=True)
bn3 = mx.sym.BatchNorm(data=conv2, fix_gamma=False, eps=2e-5)

In [86]:
mx_execute(bn3, tensors={'data': mx_inp.asnumpy()})[0].shape

(2, 64, 100, 100)

In [87]:
name = 'xxx'
bn1 = BatchNormalization(name=name + '_bn1')
conv1 = Conv2D(num_filter, kernel_size=(3, 3), strides=(1, 1), padding='valid', 
                            use_bias=False, name=name + '_conv1')
bn2 = BatchNormalization(name=name + '_bn2')
prelu1 = PReLU(name=name + 'act1')
conv2 = Conv2D(num_filter, kernel_size=(3, 3), strides=stride, padding='valid',
                                                                         use_bias=False, name=name + '_conv2')
bn3 = BatchNormalization(name=name + '_bn3')

In [88]:
tf_inp = tf_inp.astype('float32')
x = bn1(tf_inp)
x = ZeroPadding2D((1, 1))(x)
x = conv1(x)
x = bn2(x)
x = prelu1(x)
x = ZeroPadding2D((1, 1))(x)
x = conv2(x)
x = bn3(x)
x.shape

TensorShape([2, 100, 100, 64])

In [89]:
bn1 = mx.sym.BatchNorm(data=data, fix_gamma=False, eps=2e-5, name=name + '_bn1')
conv1 = Conv(data=bn1, num_filter=num_filter, kernel=(3,3), stride=(1,1), pad=(1,1),
                              no_bias=True, name=name + '_conv1')
bn2 = mx.sym.BatchNorm(data=conv1, fix_gamma=False, eps=2e-5, name=name + '_bn2')
# act1 = Act(data=bn2, act_type=act_type, name=name + '_relu1')
act1 = mx.sym.LeakyReLU(data=bn2, act_type='prelu')
conv2 = Conv(data=act1, num_filter=num_filter, kernel=(3,3), stride=stride, pad=(1,1),
                              no_bias=True, name=name + '_conv2')
bn3 = mx.sym.BatchNorm(data=conv2, fix_gamma=False, eps=2e-5,  name=name + '_bn3')

In [90]:
mx_execute(bn3, tensors={'data': mx_inp.asnumpy()})[0].shape

(2, 64, 100, 100)

In [91]:
conv1sc = Conv(data=data, num_filter=num_filter, kernel=(1,1), stride=stride, no_bias=True,
                                name=name+'_conv1sc')
shortcut = mx.sym.BatchNorm(data=conv1sc, fix_gamma=False, eps=2e-5, name=name + '_sc')

In [92]:
mx_execute(bn3, tensors={'data': mx_inp.asnumpy()})[0].shape

(2, 64, 100, 100)

In [93]:
conv1_sc = Conv2D(num_filter, kernel_size=(1, 1), strides=strides, padding='valid',
                                   use_bias=False, name=name + '_conv1sc')
bn_sc = BatchNormalization(name=name + '_sc')

In [94]:
x = conv1_sc(tf_inp)
x = bn_sc(x)
x.shape

TensorShape([2, 100, 100, 64])

In [95]:
se_pool = AveragePooling2D(name=name + '_se_pooling')
        
se_conv1 = Conv2D(num_filter//16, kernel_size=(1, 1), strides=(1, 1), padding='valid',
                       use_bias=True, name=name + '_se_conv1')
se_prelu = PReLU(name=name + '_se_act1')
se_conv2 = Conv2D(num_filter, kernel_size=(1, 1), strides=(1, 1), padding='valid',
                       use_bias=True, name=name + '_se_conv2')
se_sigmoid = Activation('sigmoid', name=name + '_se_sigmoid')

In [96]:
x = se_pool(tf_inp)
x = se_conv1(x)
x = se_prelu(x)
x = se_conv2(x)
x = se_sigmoid(x)
x.shape

TensorShape([2, 1, 1, 64])

In [97]:
conv1sc = Conv(data=data, num_filter=num_filter, kernel=(1,1), stride=stride, no_bias=True)
shortcut = mx.sym.BatchNorm(data=conv1sc, fix_gamma=False, eps=2e-5)

In [98]:
mx_execute(shortcut, tensors={'data': mx_inp.asnumpy()})[0].shape

(2, 64, 100, 100)

In [99]:
tf_conv = tf.keras.layers.Conv2D(num_filter, kernel_size=(1, 1), 
                                 strides=stride, padding='same', use_bias=False)
tf_result = tf_conv(tf_inp).numpy()

In [100]:
tf_result.shape

(2, 100, 100, 64)

In [101]:
mx_inp.shape

(2, 3, 200, 200)

In [102]:
out = bn3 + shortcut

In [103]:
mx_execute(out, tensors={'data': mx_inp.asnumpy()})[0].shape

(2, 64, 100, 100)

In [104]:
img_file = '/home/cocoza4/datasets/asr_plastic_400_400/IMG_20200222_011937.jpg'
img = mx.image.imread(img_file)
img = img.transpose((2,0,1))  # Transposing from (224, 224, 3) to (3, 224, 224)
img = img.expand_dims(axis=0).astype('float') # change the shape from (3, 224, 224) to (1, 3, 224, 224)
img.shape

(1, 3, 300, 400)

In [105]:
weights = np.random.normal(size=(5, 3, 3, 3))
tf_weights = np.transpose(weights, (1, 2, 3, 0))
weights.shape, tf_weights.shape

((5, 3, 3, 3), (3, 3, 3, 5))

In [106]:
a = mx.sym.Variable('embedding')
conv = mx.symbol.Convolution(data=a, num_filter=5, kernel=(3, 3), stride=(1,1), 
                             pad=(1, 1), name='conv', no_bias=True)
conv

<Symbol conv>

In [107]:
mx_result = mx_execute(conv, tensors={'embedding': img, 'conv_weight': weights})[0].asnumpy()
mx_result.shape

(1, 5, 300, 400)

In [121]:
def mx_residual_bock(data, num_filter, stride, name, dim_match=False):
    bn1 = mx.sym.BatchNorm(data=data, fix_gamma=False, eps=2e-5, name=name + '_bn1')
    conv1 = Conv(data=bn1, num_filter=num_filter, kernel=(3,3), stride=(1,1), pad=(1,1),
                                  no_bias=True, name=name + '_conv1')
    bn2 = mx.sym.BatchNorm(data=conv1, fix_gamma=False, eps=2e-5, name=name + '_bn2')
#     act1 = Act(data=bn2, act_type=act_type, name=name + '_relu1')
    act1 = mx.sym.LeakyReLU(data=bn2, act_type='prelu')
    conv2 = Conv(data=act1, num_filter=num_filter, kernel=(3,3), stride=stride, pad=(1,1),
                                  no_bias=True, name=name + '_conv2')
    bn3 = mx.sym.BatchNorm(data=conv2, fix_gamma=False, eps=2e-5, name=name + '_bn3')
    
    #se begin
    body = mx.sym.Pooling(data=bn3, global_pool=True, kernel=(7, 7), pool_type='avg', name=name+'_se_pool1')
    body = Conv(data=body, num_filter=num_filter//16, kernel=(1,1), stride=(1,1), pad=(0,0),
                            name=name+"_se_conv1")
#     body = Act(data=body, act_type=act_type, name=name+'_se_relu1')
    body = mx.sym.LeakyReLU(data=body, act_type='prelu')
    body = Conv(data=body, num_filter=num_filter, kernel=(1,1), stride=(1,1), pad=(0,0),
                            name=name+"_se_conv2")
    body = mx.symbol.Activation(data=body, act_type='sigmoid', name=name+"_se_sigmoid")
    bn3 = mx.symbol.broadcast_mul(bn3, body)
    #se end

    if dim_match:
        shortcut = data
    else:
        conv1sc = Conv(data=data, num_filter=num_filter, kernel=(1,1), stride=stride, no_bias=True,
                                        name=name+'_conv1sc')
        shortcut = mx.sym.BatchNorm(data=conv1sc, fix_gamma=False, eps=2e-5, name=name + '_sc')
    
    return bn3 + shortcut

In [164]:
data = mx.sym.Variable('data')
v = mx_residual_bock(data, num_filter, (2, 2), 'xxx', dim_match=False)
mx_result = mx_execute(v, tensors={'data': mx_inp.asnumpy()})[0]
mx_result.asnumpy().shape

(2, 64, 100, 100)

In [165]:
data = mx.sym.Variable('data')
v = mx_residual_bock(data, num_filter, (1, 1), 'xxx', dim_match=True)
mx_result = mx_execute(v, tensors={'data': mx_result.asnumpy()})[0].asnumpy()
mx_result.shape

(2, 64, 100, 100)

In [166]:
# data = mx.sym.Variable('data')
# v = mx_residual_bock(data, num_filter, stride, 'xxx')

# body = mx_execute(v, tensors={'data': mx_inp.asnumpy()})[0].asnumpy()
data = mx.sym.Variable('data')

body = mx.sym.BatchNorm(data=data, fix_gamma=False, eps=2e-5, name='bn1')
print('data', mx_execute(body, tensors={'data': mx_result})[0].asnumpy().shape)

body = mx.symbol.Dropout(data=body, p=0.4)
print('data', mx_execute(body, tensors={'data': mx_result})[0].asnumpy().shape)
fc1 = mx.sym.FullyConnected(data=body, num_hidden=10, name='pre_fc1')
print('data', mx_execute(fc1, tensors={'data': mx_result})[0].asnumpy().shape) # mxnet altomatically reshapes
fc1 = mx.sym.BatchNorm(data=fc1, fix_gamma=True, eps=2e-5, name='fc1')

xxx = mx_execute(fc1, tensors={'data': mx_result})[0].asnumpy()
xxx.shape

data (2, 64, 100, 100)
data (2, 64, 100, 100)
data (2, 10)


(2, 10)

In [211]:
tf_img = np.asarray(Image.open(img_file))
tf_img = np.expand_dims(tf_img, axis=0).astype('float32')
tf_img.shape

(1, 300, 400, 3)

In [263]:
tf_conv = tf.keras.layers.Conv2D(5,  weights=[tf_weights], kernel_size=(3, 3), 
                                 strides=(1, 1), padding='valid', use_bias=False)
tf_result = tf_conv(tf_img).numpy()
tf_result.shape

W0318 00:05:57.249577 140668921968448 base_layer.py:1790] Layer conv2d_23 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



(1, 298, 398, 5)

In [264]:
np.array_equal(np.transpose(tf_result, (0, 3, 1, 2)), mx_result)

False

In [ ]:
bn1 = mx.sym.BatchNorm(data=data, fix_gamma=False, eps=2e-5, momentum=bn_mom, name=name + '_bn1')
conv1 = Conv(data=bn1, num_filter=num_filter, kernel=(3,3), stride=(1,1), pad=(1,1),
                              no_bias=True, workspace=workspace, name=name + '_conv1')
bn2 = mx.sym.BatchNorm(data=conv1, fix_gamma=False, eps=2e-5, momentum=bn_mom, name=name + '_bn2')
act1 = Act(data=bn2, act_type=act_type, name=name + '_relu1')
conv2 = Conv(data=act1, num_filter=num_filter, kernel=(3,3), stride=stride, pad=(1,1),
                              no_bias=True, workspace=workspace, name=name + '_conv2')
bn3 = mx.sym.BatchNorm(data=conv2, fix_gamma=False, eps=2e-5, momentum=bn_mom, name=name + '_bn3')
if use_se:
  #se begin
  body = mx.sym.Pooling(data=bn3, global_pool=True, kernel=(7, 7), pool_type='avg', name=name+'_se_pool1')
  body = Conv(data=body, num_filter=num_filter//16, kernel=(1,1), stride=(1,1), pad=(0,0),
                            name=name+"_se_conv1", workspace=workspace)
  body = Act(data=body, act_type=act_type, name=name+'_se_relu1')
  body = Conv(data=body, num_filter=num_filter, kernel=(1,1), stride=(1,1), pad=(0,0),
                            name=name+"_se_conv2", workspace=workspace)
  body = mx.symbol.Activation(data=body, act_type='sigmoid', name=name+"_se_sigmoid")
  bn3 = mx.symbol.broadcast_mul(bn3, body)
  #se end

if dim_match:
    shortcut = data
else:
    conv1sc = Conv(data=data, num_filter=num_filter, kernel=(1,1), stride=stride, no_bias=True,
                                    workspace=workspace, name=name+'_conv1sc')
    shortcut = mx.sym.BatchNorm(data=conv1sc, fix_gamma=False, momentum=bn_mom, eps=2e-5, name=name + '_sc')
if memonger:
    shortcut._set_attr(mirror_stage='True')
return bn3 + shortcut

In [5]:
Conv2D = tf.keras.layers.Conv2D
BatchNormalization = tf.keras.layers.BatchNormalization
PReLU = tf.keras.layers.PReLU
Activation = tf.keras.layers.Activation
ZeroPadding2D = tf.keras.layers.ZeroPadding2D
Dense = tf.keras.layers.Dense
Reshape = tf.keras.layers.Reshape
Flatten = tf.keras.layers.Flatten
GlobalAveragePooling2D = tf.keras.layers.GlobalAveragePooling2D

# AveragePooling2D = tf.keras.layers.Lambda(lambda x: tf.reduce_mean(x, axis=[1, 2], keepdims=True))

In [6]:
class AveragePooling2D(tf.keras.layers.Layer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def call(self, x):
        return tf.reduce_mean(x, axis=[1, 2], keepdims=True)

In [7]:
img = Image.open('/home/cocoza4/datasets/asr_plastic/train/IMG_20200222_011937.jpg').resize((112, 112))
img = np.asarray(img).astype('float32') / 255.
inputs = np.array([img, img])
inputs.shape

(2, 112, 112, 3)

In [8]:
class ResidualBlock(tf.keras.layers.Layer):
    
    def __init__(self, filters, strides, name, shortcut=True):
        super().__init__()
        self.bn1 = BatchNormalization(name=name + '_bn1')
        self.conv1 = Conv2D(filters, kernel_size=(3, 3), strides=(1, 1), padding='valid', 
                            use_bias=False, name=name + '_conv1')
        self.bn2 = BatchNormalization(name=name + '_bn2')
        self.prelu1 = PReLU(name=name + 'act1')
        self.conv2 = Conv2D(filters, kernel_size=(3, 3), strides=strides, padding='valid',
                            use_bias=False, name=name + '_conv2')
        self.bn3 = BatchNormalization(name=name + '_bn3')
        
        self.se_pool = AveragePooling2D(name=name + '_se_pooling')
        
        self.se_conv1 = Conv2D(filters//16, kernel_size=(1, 1), strides=(1, 1), padding='valid',
                               use_bias=True, name=name + '_se_conv1')
        self.se_prelu = PReLU(name=name + '_se_act1')
        self.se_conv2 = Conv2D(filters, kernel_size=(1, 1), strides=(1, 1), padding='valid',
                               use_bias=True, name=name + '_se_conv2')
        self.se_sigmoid = Activation('sigmoid', name=name + '_se_sigmoid')

        self.shortcut = shortcut
        if shortcut:
            self.conv1_sc = Conv2D(filters, kernel_size=(1, 1), strides=strides, padding='valid',
                                   use_bias=False, name=name + '_conv1sc')
            self.bn_sc = BatchNormalization(name=name + '_sc')
        
        
    def call(self, inputs, training=False):
        # Residual Block
        x = self.bn1(inputs, training=training)
        x = ZeroPadding2D((1, 1))(x)
        x = self.conv1(x)
        x = self.bn2(x, training=training)
        x = self.prelu1(x)
        x = ZeroPadding2D((1, 1))(x)
        x = self.conv2(x)
        bn3 = self.bn3(x, training=training)
        
        # SE
        x = self.se_pool(bn3)
        x = self.se_conv1(x)
        x = self.se_prelu(x)
        x = self.se_conv2(x)
        x = self.se_sigmoid(x)
        bn3 = tf.keras.layers.multiply([bn3, x])
        
        if self.shortcut:
            shortcut = self.bn_sc(self.conv1_sc(inputs))
        else:
            shortcut = inputs
        
        return bn3 + shortcut
        
        
class StackedBlock():
    
    def __init__(self, filters, blocks, strides, name):
        self.blocks = []
        self.block = ResidualBlock(filters, strides=strides, name=name, shortcut=True)
        for i in range(blocks):
            block = ResidualBlock(filters, strides=(1, 1), name=name+f'_block{i+1}', shortcut=False)
            self.blocks.append(block)
    
    def __call__(self, x, training=False):
        x = self.block(x, training=training)
        for block in self.blocks:
            x = block(x, training=training)
        return x
    
                
        
class ResNet100(tf.keras.Model):
    
    def __init__(self, filters, blocks, embedding_size=512, name='resnet100'):
        super().__init__()
        assert len(filters) == len(blocks)
        
        self.pad = ZeroPadding2D((1, 1), name='pad')
        self.conv1 = Conv2D(64, kernel_size=(3, 3), strides=(1, 1), 
                            padding='valid', use_bias=False, name='conv1')
        self.bn1 = BatchNormalization(name='conv1_bn')
        self.prelu1 = PReLU(name='conv1_relu')
        
        self.stacks = []
        for i, f in enumerate(filters):
            name = f'conv{i+1}'
            stack = StackedBlock(f, blocks=blocks[i], strides=(2, 2), name=name)
            self.stacks.append(stack)
            
        self.bnx = BatchNormalization()
        # self.dropout = tf.keras.layers.Dropout(0.4)
        self.flatten = Flatten()
        self.dense = Dense(embedding_size, use_bias=False)
        
        self.bny = BatchNormalization(scale=False)
        
    def call(self, inputs, training=False):
        x = self.pad(inputs)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.prelu1(x)
        
        for stack in self.stacks:
            x = stack(x)
        
        print(x.shape)
        x = self.bnx(x, training=training)
        print(x.shape)
        x = self.flatten(x)
        x = self.dense(x)
        x = self.bny(x, training=training)
        
        return x

In [107]:



class ResidualBlock(tf.keras.layers.Layer):
    
    def __init__(self, kernel_size, filters, name='conv'):
        super().__init__()
        
        self.conv1 = Conv2D(filters[0], kernel_size=(1, 1), strides=(1, 1), 
                            padding='valid', use_bias=False, name=name+'_conv1')
        self.bn1 = BatchNormalization(name=name+'_bn1')
        self.relu1 = Activation('relu', name=name+'_relu1')
        
        self.conv2 = Conv2D(filters[1], kernel_size=kernel_size, # strides=strides, 
                            padding='same', use_bias=False, name=name+'_conv2')
        self.bn2 = BatchNormalization(name=name+'_bn2')
        self.relu2 = Activation('relu', name=name+'_relu2')
        
        self.conv3 = Conv2D(filters[2], kernel_size=(1, 1), strides=(1, 1), 
                            padding='valid', use_bias=False, name=name+'_conv3')
        self.bn3 = BatchNormalization(name=name+'_bn3')
        
        self.se_block = SEBlock(filters[2], name=name)
        
        self.relu3 = Activation('relu', name=name+'_relu3')

    def call(self, inputs, training=False):
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = self.relu1(x)
        
        x = self.conv2(inputs)
        x = self.bn2(x, training=training)
        x = self.relu2(x)
        
        x = self.conv3(x)
        x = self.bn3(x, training=training)
        
        x = self.se_block(x, training=training)
        
        x = tf.keras.layers.add([x, inputs])
        x = self.relu3(x)
        
        return x


class ResidualBlock(tf.keras.layers.Layer):
    
    def __init__(self, kernel_size, filters, stage, block, strides=(2, 2)):
        super().__init__()
        
        block_name = str(stage) + "_" + str(block)
        name = 'conv' + block_name
        
        self.conv1 = Conv2D(filters, kernel_size=(1, 1), strides=(1, 1), 
                            padding='valid', use_bias=False, name=name+'_conv1')
        self.bn1 = BatchNormalization(name=name+'_bn1')
        self.relu1 = Activation('relu', name=name+'_relu1')
        
        self.conv2 = Conv2D(filters[1], kernel_size=kernel_size, strides=strides, 
                            padding='same', use_bias=False, name=name+'_conv2')
        self.bn2 = BatchNormalization(name=name+'_bn2')
        self.relu2 = Activation('relu', name=name+'_relu2')
        
        self.conv3 = Conv2D(filters[2], kernel_size=(1, 1), strides=(1, 1), 
                            padding='valid', use_bias=False, name=name+'_conv3')
        self.bn3 = BatchNormalization(name=name+'_bn3')
        
        self.se_block = SEBlock(filters[2], name=name)
        
        self.conv4 = Conv2D(filters[2], kernel_size=(1, 1), strides=strides, 
                            padding='valid', use_bias=False, name=name+'_conv4')
        self.bn4 = BatchNormalization(name=name+'_bn4')
        self.relu4 = Activation('relu', name=name+'_relu4')
        
    def call(self, inputs, training=False):
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = self.relu1(x)
        
        x = self.conv2(inputs)
        x = self.bn2(x, training=training)
        x = self.relu2(x)
        
        x = self.conv3(x)
        x = self.bn3(x, training=training)
        
        x = self.se_block(x, training=training)
        
        shortcut = self.conv4(x)
        shortcut = self.bn4(shortcut, training=training)
        
        x = tf.keras.layers.add([x, shortcut])
        x = self.relu4(x)
        
        return x


class Bottleneck(tf.keras.Model):
    
    expansion = 4
    
    def __init__(self, filters, stage, block, strides=1, downsample=None):
        super().__init__()
        block_name = str(stage) + "_" + str(block)
        name = 'conv' + block_name
        
        self.conv1 = Conv2D(filters, kernel_size=1, strides=1, 
                            use_bias=False, name=name+'_conv1')
        self.bn1 = BatchNormalization(name=name+'_bn1')
        self.relu1 = Activation('relu', name=name+'_relu1')
        
        self.conv2 = Conv2D(filters, kernel_size=3, strides=strides, 
                            padding='same', use_bias=False, name=name+'_conv2')
        self.bn2 = BatchNormalization(name=name+'_bn2')
        self.relu2 = Activation('relu', name=name+'_relu2')
        
        self.conv3 = Conv2D(filters * self.expansion, kernel_size=1, strides=1, 
                            use_bias=False, name=name+'_conv3')
        self.bn3 = BatchNormalization(name=name+'_bn3')
        self.se_block = SEBlock(filters * self.expansion, name=name)
        self.relu3 = Activation('relu', name=name+'_relu3')
        
        self.downsample = downsample
        
    def call(self, inputs, training=False):
        
        if self.downsample is not None:
            identity = self.downsample(inputs, training=training)
        else:
            identity = inputs
        
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = self.relu1(x)
        
        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = self.relu2(x)
        
        x = self.conv3(x)
        x = self.bn3(x, training=training)
        
        x = self.se_block(x, training=training)
        
        x = tf.keras.layers.add([x, identity])
        
        x = self.relu3(x)
        
        return x
    
class BasicBlock(tf.keras.Model):
    
    expansion = 1
    
    def __init__(self, filters, stage, block, strides=1, downsample=None):
        super().__init__()
        block_name = str(stage) + "_" + str(block)
        name = 'conv' + block_name
        
        self.conv1 = Conv2D(filters, kernel_size=3, strides=strides, 
                            padding='same', use_bias=False, name=name+'_conv1')
        self.bn1 = BatchNormalization(name=name+'_bn1')
        self.relu1 = Activation('relu', name=name+'_relu1')
        
        self.conv2 = Conv2D(filters, kernel_size=3, strides=1, 
                            padding='same', use_bias=False, name=name+'_conv2')
        self.bn2 = BatchNormalization(name=name+'_bn2')
        self.relu2 = Activation('relu', name=name+'_relu2')
        self.se_block = SEBlock(filters, name=name)
        
        self.downsample = downsample
        
    def call(self, inputs, training=False):
        
        if self.downsample is not None:
            identity = self.downsample(inputs, training=training)
        else:
            identity = inputs
        
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = self.relu1(x)
        
        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = self.se_block(x, training=training)
        
        x = tf.keras.layers.add([x, identity])
        x = self.relu2(x)
        
        return x
        
class SEBlock(tf.keras.layers.Layer):
    
    def __init__(self, filters, name='conv'):
        super().__init__()
#         self.se_pool = GlobalAveragePooling2D(name=name+'_se_pool')
#         self.se_dense1 = Dense(filters // 16, activation='relu', name=name+'_se_fc1') # squeeze
#         self.se_dense2 = Dense(filters, activation='sigmoid', name=name+'_se_fc2') # excite
        self.pool = AveragePooling2D(name=name+'_se_pooling')
        
        self.conv1 = Conv2D(filters//16, kernel_size=(1, 1), strides=(1, 1), padding='valid',
                               use_bias=True, name=name + '_se_conv1')
        self.relu1 = Activation('relu', name=name + '_se_relu1')
        self.conv2 = Conv2D(filters, kernel_size=(1, 1), strides=(1, 1), padding='valid',
                               use_bias=True, name=name + '_se_conv2')
        self.sigmoid = Activation('sigmoid', name=name + '_se_sigmoid')
#         self.reshape = Reshape([1, 1, filters])
        
    def call(self, inputs):
        x = self.pool(inputs)
#         x = self.se_dense1(x)
#         x = self.se_dense2(x)
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.sigmoid(x)
#         x = self.reshape(x)
        x = tf.keras.layers.multiply([x, inputs])
        return x
    

class SEResnet(tf.keras.Model):
    
    def __init__(self, block_type, blocks, name='se-resnet50'):
        super().__init__()
        
        self.conv1 = Conv2D(64, kernel_size=3, strides=1, 
                            padding='valid', use_bias=False, name='conv1')
        self.bn1 = BatchNormalization(name='conv1_bn')
        self.relu1 = Activation('relu', name='conv1_relu')
        
        self.stage2 = self.build_stage(64, block_type, blocks[0], strides=1, stage=2)
        self.stage3 = self.build_stage(128, block_type, blocks[1], strides=2, stage=3)
        self.stage4 = self.build_stage(256, block_type, blocks[2], strides=2, stage=4)
        self.stage5 = self.build_stage(512, block_type, blocks[3], strides=2, stage=5)

    def build_stage(self, filters, block_type, n_blocks, strides, stage):
        
        downsample = tf.keras.Sequential([
            Conv2D(filters * block_type.expansion, kernel_size=1, strides=strides, name=f'conv{stage}_0_conv0'),
            BatchNormalization(name=f'conv{stage}_0_bn0')
        ])
        
        blocks = [block_type(filters, stage, block=1, strides=strides, downsample=downsample)]
        for i in range(1, n_blocks):
            blocks.append(block_type(filters, stage, block=i+1, strides=1))
            
        return tf.keras.Sequential(blocks)
            
    def call(self, inputs, training=False):
        x = ZeroPadding2D()(inputs)
        x = self.conv1(x)
        x = self.bn1(x, training=training)
        x = self.relu1(x)
        
        x = self.stage2(x, training=training)
        x = self.stage3(x, training=training)
        x = self.stage4(x, training=training)
        x = self.stage5(x, training=training)
        
        return x
    
    

In [108]:
se_resnet50 = SEResnet(BasicBlock, blocks=[3, 4, 6, 3])

In [110]:
se_resnet50.summary()

Model: "se_resnet_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               multiple                  1728      
_________________________________________________________________
conv1_bn (BatchNormalization multiple                  256       
_________________________________________________________________
conv1_relu (Activation)      multiple                  0         
_________________________________________________________________
sequential_123 (Sequential)  multiple                  228876    
_________________________________________________________________
sequential_125 (Sequential)  multiple                  1127584   
_________________________________________________________________
sequential_127 (Sequential)  multiple                  6880096   
_________________________________________________________________
sequential_129 (Sequential)  multiple                 

In [109]:
out = se_resnet50(inputs)
out.shape

TensorShape([2, 14, 14, 512])

In [56]:
r50 = tf.keras.applications.ResNet50(input_shape=(112, 112, 3), include_top=False)

In [63]:
r50(inputs).shape

TensorShape([2, 4, 4, 2048])

In [69]:
r50.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 112, 112, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 118, 118, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 56, 56, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 56, 56, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [79]:
Flatten()(out)

<tf.Tensor: shape=(2, 401408), dtype=float32, numpy=
array([[0.00077106, 0.        , 0.        , ..., 0.01320089, 0.00525532,
        0.00012029],
       [0.00077106, 0.        , 0.        , ..., 0.01320088, 0.00525532,
        0.00012029]], dtype=float32)>

In [66]:
se_resnet50.summary()

Model: "se_resnet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               multiple                  1728      
_________________________________________________________________
conv1_bn (BatchNormalization multiple                  256       
_________________________________________________________________
conv1_relu (Activation)      multiple                  0         
_________________________________________________________________
sequential_31 (Sequential)   multiple                  244272    
_________________________________________________________________
sequential_33 (Sequential)   multiple                  1360512   
_________________________________________________________________
sequential_35 (Sequential)   multiple                  7912832   
_________________________________________________________________
sequential_37 (Sequential)   multiple                  16

In [ ]:
net = tl.layers.ReshapeLayer(net, shape=[-1, net_shape[1]*net_shape[2]*net_shape[3]], name='E_Reshapelayer')

In [67]:
se_resnet101 = SEResnet(blocks=[3, 4, 23, 3])
se_resnet101(inputs)

<tf.Tensor: shape=(2, 14, 14, 2048), dtype=float32, numpy=
array([[[[2.60395766e-03, 7.80167244e-03, 0.00000000e+00, ...,
          7.31014507e-03, 3.11156124e-04, 8.52757017e-04],
         [4.94300108e-03, 1.35528538e-02, 0.00000000e+00, ...,
          1.37080923e-02, 1.59830949e-03, 1.94010744e-03],
         [6.63624052e-03, 1.71622336e-02, 0.00000000e+00, ...,
          1.51520651e-02, 1.77050428e-03, 5.45564853e-03],
         ...,
         [3.68480105e-04, 1.67093473e-03, 0.00000000e+00, ...,
          1.40380450e-02, 1.27022795e-03, 4.02054936e-03],
         [1.45441769e-02, 1.36701884e-02, 0.00000000e+00, ...,
          2.05905568e-02, 2.01677717e-03, 3.40851257e-04],
         [4.45851078e-03, 3.82111734e-03, 0.00000000e+00, ...,
          1.45491855e-02, 7.22471275e-04, 3.08329007e-03]],

        [[4.30447329e-03, 1.52721554e-02, 1.55494025e-04, ...,
          9.80110839e-03, 3.12765478e-05, 2.81450595e-03],
         [7.53156189e-03, 2.03183684e-02, 0.00000000e+00, ...,
        

In [68]:
se_resnet101.summary()

Model: "se_resnet_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               multiple                  1728      
_________________________________________________________________
conv1_bn (BatchNormalization multiple                  256       
_________________________________________________________________
conv1_relu (Activation)      multiple                  0         
_________________________________________________________________
sequential_39 (Sequential)   multiple                  244272    
_________________________________________________________________
sequential_41 (Sequential)   multiple                  1360512   
_________________________________________________________________
sequential_43 (Sequential)   multiple                  29203904  
_________________________________________________________________
sequential_45 (Sequential)   multiple                  

In [29]:
filters = [64, 128, 256, 512]
blocks = [2, 12, 29, 2]
input_shape = (None, 112, 112, 3)


In [30]:
resnet100 = ResNet100(filters, blocks)
resnet100.compile(optimizer='adam',
                  loss='categorical_crossentropy')

In [31]:
pred = resnet100.predict(inputs)
pred.shape

(None, 7, 7, 512)
(None, 7, 7, 512)
(None, 7, 7, 512)
(None, 7, 7, 512)


(2, 512)

In [ ]:
units, version_unit

In [40]:
filter_list

[64, 64, 128, 256, 512]

In [41]:
units = [2, 12, 29, 2]

In [42]:
tf.keras.layers.Conv2D(filter_list, kernel_size=(3, 3), strides=(1, 1), padding='same', use_bias=False)
tf.keras.layers.BatchNormalization()
tf.keras.layers.PReLU()

In [ ]:
for i in range(num_stages):
    body = residual_unit_v1_L(body, filter_list[i+1], (2, 2), False,
        name='stage%d_unit%d' % (i + 1, 1), bottle_neck=bottle_neck, **kwargs)
    
    for j in range(units[i]-1):
        body = residual_unit(body, filter_list[i+1], (1,1), True, name='stage%d_unit%d' % (i+1, j+2),
          bottle_neck=bottle_neck, **kwargs)

In [ ]:
residual_unit_v1_L(data, num_filter, stride, dim_match, name, bottle_neck, **kwargs)

In [7]:
#se begin
          body = mx.sym.Pooling(data=bn2, global_pool=True, kernel=(7, 7), pool_type='avg', name=name+'_se_pool1')
          body = Conv(data=body, num_filter=num_filter//16, kernel=(1,1), stride=(1,1), pad=(0,0),
                                    name=name+"_se_conv1", workspace=workspace)
          body = Act(data=body, act_type=act_type, name=name+'_se_relu1')
          body = Conv(data=body, num_filter=num_filter, kernel=(1,1), stride=(1,1), pad=(0,0),
                                    name=name+"_se_conv2", workspace=workspace)
          body = mx.symbol.Activation(data=body, act_type='sigmoid', name=name+"_se_sigmoid")
          bn2 = mx.symbol.broadcast_mul(bn2, body)
          #se end

xxx


In [8]:
def residual_unit_v1_L(name, filters, stride):
    tf.keras.layers.ZeroPadding2D((1, 1))
    tf.keras.layers.Conv2D(filters, kernel_size=(3, 3), strides=(1, 1), padding='same', use_bias=False)
    tf.keras.layers.BatchNormalization()
    tf.keras.layers.PReLU()
    
    tf.keras.layers.ZeroPadding2D((1, 1))
    tf.keras.layers.Conv2D(filters, kernel_size=(3, 3), strides=(1, 1), padding='same', use_bias=False)
    bn2 = tf.keras.layers.BatchNormalization()
    
    # begin se
    tf.keras.layers.AveragePooling2D(pool_size=(7, 7))
    tf.keras.layers.Conv2D(filters//16, kernel_size=(1, 1), strides=(1, 1), padding='same', use_bias=False)
    tf.keras.layers.PReLU()
    
    tf.keras.layers.Conv2D(filters, kernel_size=(1, 1), strides=(1, 1), padding='same', use_bias=False)
    x = tf.keras.layers.PReLU()
    
    bn2 = bn2 * x
    # end se
    
    tf.keras.layers.Conv2D(filters, kernel_size=(1, 1), strides=stride, padding='same', use_bias=False)
    shortcut = tf.keras.layers.BatchNormalization()
    
    x = bn2 + shortcut
    return x


In [ ]:
conv1 = Conv(data=data, num_filter=num_filter, kernel=(3,3), stride=(1,1), pad=(1,1),
                              no_bias=True, workspace=workspace, name=name + '_conv1')
bn1 = mx.sym.BatchNorm(data=conv1, fix_gamma=False, momentum=bn_mom, eps=2e-5, name=name + '_bn1')
act1 = Act(data=bn1, act_type=act_type, name=name + '_relu1')
conv2 = Conv(data=act1, num_filter=num_filter, kernel=(3,3), stride=stride, pad=(1,1),
                              no_bias=True, workspace=workspace, name=name + '_conv2')
bn2 = mx.sym.BatchNorm(data=conv2, fix_gamma=False, momentum=bn_mom, eps=2e-5, name=name + '_bn2')
if use_se:
  #se begin
  body = mx.sym.Pooling(data=bn2, global_pool=True, kernel=(7, 7), pool_type='avg', name=name+'_se_pool1')
  body = Conv(data=body, num_filter=num_filter//16, kernel=(1,1), stride=(1,1), pad=(0,0),
                            name=name+"_se_conv1", workspace=workspace)
  body = Act(data=body, act_type=act_type, name=name+'_se_relu1')
  body = Conv(data=body, num_filter=num_filter, kernel=(1,1), stride=(1,1), pad=(0,0),
                            name=name+"_se_conv2", workspace=workspace)
  body = mx.symbol.Activation(data=body, act_type='sigmoid', name=name+"_se_sigmoid")
  bn2 = mx.symbol.broadcast_mul(bn2, body)
  #se end

In [ ]:
def residual_unit_v1_L(data, num_filter, stride, dim_match, name, bottle_neck, **kwargs):
    """Return ResNet Unit symbol for building ResNet
    Parameters
    ----------
    data : str
        Input data
    num_filter : int
        Number of output channels
    bnf : int
        Bottle neck channels factor with regard to num_filter
    stride : tuple
        Stride used in convolution
    dim_match : Boolean
        True means channel number between input and output is the same, otherwise means differ
    name : str
        Base name of the operators
    workspace : int
        Workspace used in convolution operator
    """
    use_se = kwargs.get('version_se', 1)
    bn_mom = kwargs.get('bn_mom', 0.9)
    workspace = kwargs.get('workspace', 256)
    memonger = kwargs.get('memonger', False)
    act_type = kwargs.get('version_act', 'prelu')
    #print('in unit1')
    if bottle_neck:
        conv1 = Conv(data=data, num_filter=int(num_filter*0.25), kernel=(1,1), stride=(1,1), pad=(0,0),
                                   no_bias=True, workspace=workspace, name=name + '_conv1')
        bn1 = mx.sym.BatchNorm(data=conv1, fix_gamma=False, eps=2e-5, momentum=bn_mom, name=name + '_bn1')
        act1 = Act(data=bn1, act_type=act_type, name=name + '_relu1')
        conv2 = Conv(data=act1, num_filter=int(num_filter*0.25), kernel=(3,3), stride=(1,1), pad=(1,1),
                                   no_bias=True, workspace=workspace, name=name + '_conv2')
        bn2 = mx.sym.BatchNorm(data=conv2, fix_gamma=False, eps=2e-5, momentum=bn_mom, name=name + '_bn2')
        act2 = Act(data=bn2, act_type=act_type, name=name + '_relu2')
        conv3 = Conv(data=act2, num_filter=num_filter, kernel=(1,1), stride=stride, pad=(0,0), no_bias=True,
                                   workspace=workspace, name=name + '_conv3')
        bn3 = mx.sym.BatchNorm(data=conv3, fix_gamma=False, eps=2e-5, momentum=bn_mom, name=name + '_bn3')

        if use_se:
          #se begin
          body = mx.sym.Pooling(data=bn3, global_pool=True, kernel=(7, 7), pool_type='avg', name=name+'_se_pool1')
          body = Conv(data=body, num_filter=num_filter//16, kernel=(1,1), stride=(1,1), pad=(0,0),
                                    name=name+"_se_conv1", workspace=workspace)
          body = Act(data=body, act_type=act_type, name=name+'_se_relu1')
          body = Conv(data=body, num_filter=num_filter, kernel=(1,1), stride=(1,1), pad=(0,0),
                                    name=name+"_se_conv2", workspace=workspace)
          body = mx.symbol.Activation(data=body, act_type='sigmoid', name=name+"_se_sigmoid")
          bn3 = mx.symbol.broadcast_mul(bn3, body)
          #se end

        if dim_match:
            shortcut = data
        else:
            conv1sc = Conv(data=data, num_filter=num_filter, kernel=(1,1), stride=stride, no_bias=True,
                                            workspace=workspace, name=name+'_conv1sc')
            shortcut = mx.sym.BatchNorm(data=conv1sc, fix_gamma=False, eps=2e-5, momentum=bn_mom, name=name + '_sc')
        if memonger:
            shortcut._set_attr(mirror_stage='True')
        return Act(data=bn3 + shortcut, act_type=act_type, name=name + '_relu3')
    else:
        conv1 = Conv(data=data, num_filter=num_filter, kernel=(3,3), stride=(1,1), pad=(1,1),
                                      no_bias=True, workspace=workspace, name=name + '_conv1')
        bn1 = mx.sym.BatchNorm(data=conv1, fix_gamma=False, momentum=bn_mom, eps=2e-5, name=name + '_bn1')
        act1 = Act(data=bn1, act_type=act_type, name=name + '_relu1')
        conv2 = Conv(data=act1, num_filter=num_filter, kernel=(3,3), stride=stride, pad=(1,1),
                                      no_bias=True, workspace=workspace, name=name + '_conv2')
        bn2 = mx.sym.BatchNorm(data=conv2, fix_gamma=False, momentum=bn_mom, eps=2e-5, name=name + '_bn2')
        if use_se:
          #se begin
          body = mx.sym.Pooling(data=bn2, global_pool=True, kernel=(7, 7), pool_type='avg', name=name+'_se_pool1')
          body = Conv(data=body, num_filter=num_filter//16, kernel=(1,1), stride=(1,1), pad=(0,0),
                                    name=name+"_se_conv1", workspace=workspace)
          body = Act(data=body, act_type=act_type, name=name+'_se_relu1')
          body = Conv(data=body, num_filter=num_filter, kernel=(1,1), stride=(1,1), pad=(0,0),
                                    name=name+"_se_conv2", workspace=workspace)
          body = mx.symbol.Activation(data=body, act_type='sigmoid', name=name+"_se_sigmoid")
          bn2 = mx.symbol.broadcast_mul(bn2, body)
          #se end

        if dim_match:
            shortcut = data
        else:
            conv1sc = Conv(data=data, num_filter=num_filter, kernel=(1,1), stride=stride, no_bias=True,
                                            workspace=workspace, name=name+'_conv1sc')
            shortcut = mx.sym.BatchNorm(data=conv1sc, fix_gamma=False, momentum=bn_mom, eps=2e-5, name=name + '_sc')
        if memonger:
            shortcut._set_attr(mirror_stage='True')
        return Act(data=bn2 + shortcut, act_type=act_type, name=name + '_relu3')

In [ ]:
body = Conv(data=body, num_filter=filter_list[0], kernel=(3,3), stride=(1,1), pad=(1, 1),
                                no_bias=True, name="conv0", workspace=workspace)
      body = mx.sym.BatchNorm(data=body, fix_gamma=False, eps=2e-5, momentum=bn_mom, name='bn0')
      body = Act(data=body, act_type=act_type, name='relu0')